Download heart disease dataset heart.csv in [Exercise](https://github.com/codebasics/py/tree/master/ML/19_Bagging/Exercise) folder and do following, (credits of dataset:  https://www.kaggle.com/fedesoriano/heart-failure-prediction)

1. Load heart disease dataset in pandas dataframe
1. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
1. Convert text columns to numbers using label encoding and one hot encoding
1. Apply scaling
1. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
1. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
1. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [1]:
import pandas as pd

df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


### 2.Remove outliers using Z score

In [2]:
u_limit = df.Cholesterol.mean() + 3*df.Cholesterol.std()
u_limit

526.9519979267629

In [3]:
df1 = df[  df.Cholesterol <= u_limit]
df1.shape

(915, 12)

In [4]:
df2 = df1[df1.Oldpeak <= (df1.Oldpeak.mean()+3*df1.Oldpeak.std())]
df2.shape

(909, 12)

In [5]:
df3 = df2[df2.RestingBP <= (df2.RestingBP.mean()+3*df2.RestingBP.std())]
df3.shape
# other features doesn't have outliers

(902, 12)

### 3.Convert text columns to numbers using label encoding and one hot encoding

In [6]:
df.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [7]:
df.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [8]:
df.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [9]:
df.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [10]:
# Label encoding
df4 = df3.copy()
df4.ExerciseAngina.replace(
    {
        'N': 0,
        'Y': 1
    },
    inplace=True)

df4.ST_Slope.replace(
    {
        'Down': 1,
        'Flat': 2,
        'Up': 3
    },
    inplace=True
)

df4.RestingECG.replace(
    {
        'Normal': 1,
        'ST': 2,
        'LVH': 3
    },
    inplace=True)

df4.head()

/var/folders/dq/zrk1d71s0gb3ngg449qdwx3h0000gp/T/ipykernel_2579/2095572947.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df4.ExerciseAngina.replace(
/var/folders/dq/zrk1d71s0gb3ngg449qdwx3h0000gp/T/ipykernel_2579/2095572947.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df4.ExerciseAngina

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,3,0
1,49,F,NAP,160,180,0,1,156,0,1.0,2,1
2,37,M,ATA,130,283,0,2,98,0,0.0,3,0
3,48,F,ASY,138,214,0,1,108,1,1.5,2,1
4,54,M,NAP,150,195,0,1,122,0,0.0,3,0


In [11]:
# One Hot encoding
df5 = pd.get_dummies(df4, drop_first=True)
df5.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,0,True,True,False,False
1,49,160,180,0,1,156,0,1.0,2,1,False,False,True,False
2,37,130,283,0,2,98,0,0.0,3,0,True,True,False,False
3,48,138,214,0,1,108,1,1.5,2,1,False,False,False,False
4,54,150,195,0,1,122,0,0.0,3,0,True,False,True,False


In [12]:
X = df5.drop("HeartDisease",axis='columns')
y = df5.HeartDisease

X.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,True,True,False,False
1,49,160,180,0,1,156,0,1.0,2,False,False,True,False
2,37,130,283,0,2,98,0,0.0,3,True,True,False,False
3,48,138,214,0,1,108,1,1.5,2,False,False,False,False
4,54,150,195,0,1,122,0,0.0,3,True,False,True,False


### 4.Apply scaling

In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42896269,  0.46089071,  0.85238015, ...,  2.06757196,
        -0.53547478, -0.22914788],
       [-0.47545956,  1.5925728 , -0.16132855, ..., -0.4836591 ,
         1.86750159, -0.22914788],
       [-1.74679706, -0.10495034,  0.79657967, ...,  2.06757196,
        -0.53547478, -0.22914788],
       ...,
       [ 0.37209878, -0.10495034, -0.61703246, ..., -0.4836591 ,
        -0.53547478, -0.22914788],
       [ 0.37209878, -0.10495034,  0.35947592, ...,  2.06757196,
        -0.53547478, -0.22914788],
       [-1.64085227,  0.3477225 , -0.20782894, ..., -0.4836591 ,
         1.86750159, -0.22914788]])

### 5.Build a classification model using SVM. 
Use standalone model as well as Bagging model and check if you see any difference in the performance.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=30)

In [15]:
X_train.shape

(721, 13)

<h3>Train using stand alone model</h3>

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

scores = cross_val_score(SVC(), X, y, cv=5)
scores

array([0.62430939, 0.79558011, 0.68888889, 0.72777778, 0.61666667])

In [20]:
scores.mean()

0.6906445672191528

<h3>Train using Bagging</h3>

In [25]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.869625520110957

### 6.Build a classification model using decision tree . 
Use standalone model as well as Bagging model and check if you see any difference in the performance.

<h3>Train using stand alone model</h3>

In [26]:
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.75690608, 0.77348066, 0.76666667, 0.7       , 0.61666667])

In [27]:
scores.mean()

0.7227440147329649

<h3>Train using Bagging</h3>

In [28]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.8474341192787794

### 7.Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. 
Use internet to figure out in what conditions bagging works the best.

In [30]:
tree_improv = 0.84 - 0.72
tree_improv

0.12

In [31]:
svc_improv = 0.86 - 0.69
svc_improv

0.17000000000000004

Bagging is usually helpful with “unstable” classification algorithms like neural networks and decision trees.

Random Forest uses bagging underneath.